# One_sentiment_analysis_model

### Exploração e limpeza dos dados (EDA)

In [2]:
# instalar os pacotes ncessarios no ambiente virtual
%pip install pandas nltk spacy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
# baixa e instala o modelo small do spacy para português (tokenização, POS, lematização, etc.), que depois pode ser carregado com `spacy.load("pt_core_news_sm")`.
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.5 MB/s  0:00:00eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
# dataset inicial? https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis/data
# este dataset foi baixado manualmente e colocado no diretorio `src/data/`

In [4]:
# inicializacao das bibliotecas
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
import spacy

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score


In [12]:
# carregando o dataset original
df = pd.read_csv('../data/NoThemeTweets.csv')
df.tail()

,id,tweet_text,tweet_date,sentiment,query_used
785809,1050705141207367680,Acordar 8 horas é tão bom :),Fri Oct 12 11:10:01 +0000 2018,Positivo,:)
785810,1050706655049109504,"@mayckcunha Olá, Mayck. Você já é cliente Clar...",Fri Oct 12 11:16:02 +0000 2018,Positivo,:)
785811,1050705846907392005,Opa tava na merda mm e fiquei logo mais feliz ...,Fri Oct 12 11:12:49 +0000 2018,Positivo,:)
785812,1050705490232127489,@andrebraga2806 Foi como a tua lealdade :),Fri Oct 12 11:11:24 +0000 2018,Positivo,:)
785813,1050704920922521601,Feliz dia das crianças!! De hoje e de ontem......,Fri Oct 12 11:09:08 +0000 2018,Positivo,:)


In [13]:
# removendo colunas desnessarias e limitando o dataset para 50.000 amostras
df = df.drop(["id", "tweet_date"], axis=1)
df = df[:50000]
df.head()

,tweet_text,sentiment,query_used
0,@Tixaa23 14 para eu ir :),Positivo,:)
1,@drexalvarez O meu like eu já dei na época :),Positivo,:)
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo,:)
3,:D que lindo dia !,Positivo,:)
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo,:)


### Transformação dos textos em números com TF-IDF

In [14]:
# converte colunas categóricas em valores inteiros com LabelEncoder:
# mapeia cada rótulo de texto em um inteiro (ex.: 'Positivo' -> 0, 'Negativo' -> 1)
lb = LabelEncoder()

df["sentiment"] = lb.fit_transform(df["sentiment"])
df["query_used"] = lb.fit_transform(df["query_used"])


In [15]:
df.shape

(50000, 3)

In [16]:
# verificando se ha valores nulos no dataset
df.isnull().sum()

tweet_text    0
sentiment     0
query_used    0
dtype: int64

In [20]:
# carrega o modelo do spacy para portugues e cria o objeto `nlp` (tokenizador, POS, lematizador, dependências etc.). Esse `nlp` é usado para processar textos (ex.: `nlp(text)`).
nlp = spacy.load("pt_core_news_sm")

In [17]:
# baixando recursos do nltk (se ainda nao tiverem sido baixados)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
# definindo as stop words em portugues
stop_words = nltk.corpus.stopwords.words('portuguese')

In [21]:
# tokenizando os tweets com spacy
tweets = df["tweet_text"]
tokenization = [nlp(text.lower()) for text in tweets]

In [22]:
import nltk
# baixando recursos do nltk (se ainda nao tiverem sido baixados) em um diretorio especifico
nltk.download('wordnet', download_dir='../data/kaggle/working/nltk_data')
nltk.data.path.append("../data/kaggle/working/nltk_data")


[nltk_data] Downloading package wordnet to
[nltk_data]     ../data/kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# aplicando stemming e removendo stop words, pontuações, mentions e links
new_tweets = []
for phrase in tokenization:
  new_phrase = ""
  for token in phrase:
    if not str(token) in stop_words and not token.is_punct and "@" not in str(token) and "http" not in str(token):
      new_phrase += ps.stem(str(token)) + " "
  new_tweets.append(new_phrase[:-1])


In [24]:
# atualizando a coluna tweet_text com os tweets processados
df["tweet_text"] = new_tweets

In [25]:
df.head()

,tweet_text,sentiment,query_used
0,14 ir,1,1
1,like dei época,1,1
2,queria conseguir comer alguma coisa pra poder ...,1,1
3,d lindo dia,1,1
4,pq pr jeito!!é oferta ha q aproveitar :p,1,1


### Treinamento de modelo supervisionado (ex.: Logistic Regression, Naive Bayes)

In [26]:
# criando vetorizadores unigrama com CountVectorizer e TfidfVectorizer
vect_uni_cv = CountVectorizer(ngram_range=(1,1), stop_words=stop_words)
text_vect_uni_cv = vect_uni_cv.fit_transform(df["tweet_text"])

# dividindo os dados em treino e teste (80% treino, 20% teste)
X_trainUCV, X_testUCV, y_trainUCV, y_testUCV = train_test_split(text_vect_uni_cv, df["sentiment"], test_size=0.2, random_state=42)

# criando vetorizadores unigrama com TfidfVectorizer
vect_uni_idf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l2', stop_words=stop_words)
text_vect_uni_idf = vect_uni_idf.fit_transform(df["tweet_text"])

# dividindo os dados em treino e teste (80% treino, 20% teste)
X_trainUIDF, X_testUIDF, y_trainUIDF, y_testUIDF = train_test_split(text_vect_uni_idf, df["sentiment"], test_size=0.2, random_state=42)


In [27]:
# treinando e avaliando o modelo Random Forest com CountVectorizer
rfcUCV = RandomForestClassifier()

rfcUCV.fit(X_trainUCV, y_trainUCV)
y_predUCV = rfcUCV.predict(X_testUCV)

acUCV = accuracy_score(y_testUCV, y_predUCV)

print(f'Score Count Vectorizer Random Forest: {acUCV*100:.2f}%')


Score Count Vectorizer Random Forest: 99.99%


In [28]:
# treinando e avaliando o modelo Random Forest com TfidfVectorizer
rfcidf = RandomForestClassifier()

rfcidf.fit(X_trainUIDF, y_trainUIDF)
y_predidf = rfcidf.predict(X_testUIDF)

acidf = accuracy_score(y_testUIDF, y_predidf)

print(f'Score TFIDF Random Forest: {acidf*100:.2f}%')


Score TFIDF Random Forest: 99.99%


In [29]:
# treinando e avaliando o modelo Decision Tree com CountVectorizer
dtrUVC = DecisionTreeClassifier()

dtrUVC.fit(X_trainUCV, y_trainUCV)

acUCV = dtrUVC.score(X_testUCV, y_testUCV)

print(f'Score Count Vectorizer Decision Tree Classifier: {acUCV*100:.2f}%')


Score Count Vectorizer Decision Tree Classifier: 99.95%


In [30]:
# treinando e avaliando o modelo Decision Tree com TfidfVectorizer
dtridf = DecisionTreeClassifier()

dtridf.fit(X_trainUIDF, y_trainUIDF)

acidf = dtridf.score(X_testUCV, y_testUCV)

print(f'Score TFIDF Decision Tree Classifier: {acidf*100:.2f}%')


Score TFIDF Decision Tree Classifier: 99.76%


In [32]:
# treinando e avaliando o modelo usando Naive Bayes com CountVectorizer
from sklearn.naive_bayes import MultinomialNB
mnbUCV = MultinomialNB()
mnbUCV.fit(X_trainUCV, y_trainUCV)
y_predNB = mnbUCV.predict(X_testUCV)
acNB = accuracy_score(y_testUCV, y_predNB)
print(f'Score Count Vectorizer Naive Bayes: {acNB*100:.2f}%')

Score Count Vectorizer Naive Bayes: 99.80%


In [33]:
# treinando e avaliando o modelo usando Naive Bayes com TfidfVectorizer
mnbIDF = MultinomialNB()
mnbIDF.fit(X_trainUIDF, y_trainUIDF)
y_predNBIDF = mnbIDF.predict(X_testUIDF)
acNBIDF = accuracy_score(y_testUIDF, y_predNBIDF)
print(f'Score TFIDF Naive Bayes: {acNBIDF*100:.2f}%')

Score TFIDF Naive Bayes: 99.99%


### Métricas de desempenho (Acurácia, Precisão, Recall, F1-score)

In [ ]:
# TODO

### Serialização do modelo (joblib/pickle)

In [34]:
# Serializacao do modelo treinado com Random Forest e CountVectorizer usando pickle para uso posterior sem necessidade de re-treinamento
import pickle
with open('../models/rfcUCV_model.pkl', 'wb') as f:
    pickle.dump(rfcUCV, f)

In [35]:
# Serializacao do modelo treinado com Random Forest e TfidfVectorizer usando pickle para uso posterior sem necessidade de re-treinamento
with open('../models/rfcUIDF_model.pkl', 'wb') as f:
    pickle.dump(rfcidf, f)

In [36]:
# Serializacao do modelo treinado com Decision Tree e CountVectorizer usando pickle para uso posterior sem necessidade de re-treinamento
with open('../models/dtrUCV_model.pkl', 'wb') as f:
    pickle.dump(dtrUVC, f)

In [37]:
# Serializacao do modelo treinado com Decision Tree e TfidfVectorizer usando pickle para uso posterior sem necessidade de re-treinamento
with open('../models/dtrUIDF_model.pkl', 'wb') as f:
    pickle.dump(dtridf, f)

In [38]:
# Serializacao do modelo treinado com Naive Bayes e CountVectorizer usando pickle para uso posterior sem necessidade de re-treinamento
with open('../models/mnbUCV_model.pkl', 'wb') as f:
    pickle.dump(mnbUCV, f)

In [39]:
# Serializacao do modelo treinado com Naive Bayes e CountVectorizer usando pickle para uso posterior sem necessidade de re-treinamento
with open('../models/mnbUCV_model.pkl', 'wb') as f:
    pickle.dump(mnbUCV, f)